In [11]:
!pip install transformers torch feedparser requests

import feedparser
import requests
import torch
from transformers import pipeline
from datetime import datetime
import os
import glob
from google.colab import files  # For downloads

In [12]:
FEED_URL = "https://api.substack.com/feed/podcast/623622/s/13426.rss"
PREVIEW_SECS = 60  # Skirt paywalls; bump for full (paid auth later)

feed = feedparser.parse(FEED_URL)
episodes = []

for entry in feed.entries[:3]:  # Last 3 for low entropy
    title = entry.title
    date = entry.published if 'published' in entry else str(datetime.now())
    desc = entry.summary[:100] + "..." if entry.summary else ""
    audio_url = next((enc.href for enc in entry.enclosures if enc.type == 'audio/mpeg'), None)

    if audio_url:
        temp_file = f"temp_{title[:20]}.mp3"
        resp = requests.get(audio_url, stream=True)
        if resp.status_code == 200:
            with open(temp_file, 'wb') as f:
                chunk_size = 8192
                total_bytes = 0
                for chunk in resp.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        total_bytes += len(chunk)
                        if total_bytes > PREVIEW_SECS * 16000:  # Rough byte est for 16kHz
                            break
            print(f"Downloaded preview: {title}")
        else:
            print(f"Skip: {title} (HTTP {resp.status_code})")
            continue

    # Transcribe next cell
    episodes.append({'title': title, 'date': date, 'desc': desc, 'temp_file': temp_file})

print(f"Queued {len(episodes)} episodes for transcription.")

Downloaded preview: Epilogue — The Sea and the Stone
Downloaded preview: The Odyssey: The Journey Home
Downloaded preview: Bitcoin: The Stateless Crown
Queued 3 episodes for transcription.


In [13]:
# Load Whisper (base: fast, accurate enough for previews)
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base")

transcripts = []
for ep in episodes:
    if 'temp_file' in ep and os.path.exists(ep['temp_file']):
        # Enable long-form for >30s clips (timestamps auto-ignored for text)
        transcript = transcriber(ep['temp_file'], return_timestamps=True)['text']

        # Simple Prune: Filter short words, cap length (GCI proxy: >0.7 words/util)
        words = [w for w in transcript.split() if len(w) > 3]
        pruned = ' '.join(words[:250])  # Entropy cap; tie to S(ρ) via numpy later

        ep['transcript'] = pruned
        ep['coherence_proxy'] = len(words) / 250  # Quick >0.7 check

        # VOW Flag
        if ep['coherence_proxy'] < 0.7:
            print(f"Low resonance: {ep['title']} → recalibrate_equilibria")
        else:
            print(f"Pruned: {ep['title']} ({len(words)} words)")

        # Clean temp
        os.remove(ep['temp_file'])

    transcripts.append(ep)

# Output as JSON (downloadable)
import json
output = json.dumps(transcripts, indent=2, default=str)
with open('podcast_transcripts.json', 'w') as f:
    f.write(output)
files.download('podcast_transcripts.json')  # Auto-downloads to your machine

print("Fusion-ready: transcripts.json downloaded. Feed to diversity_entropy_v6.py!")

Device set to use cpu


Low resonance: Epilogue — The Sea and the Stone → recalibrate_equilibria
Low resonance: The Odyssey: The Journey Home → recalibrate_equilibria
Low resonance: Bitcoin: The Stateless Crown → recalibrate_equilibria


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fusion-ready: transcripts.json downloaded. Feed to diversity_entropy_v6.py!


In [14]:
import requests

# Auto-discover .txt files via GitHub API (public repo)
api_url = "https://api.github.com/repos/babyblueviper1/Viper-Stack-Omega/contents/narratives/Waternova/chapters"
resp = requests.get(api_url)
if resp.status_code == 200:
    files_list = [item['name'] for item in resp.json() if item['name'].endswith('.txt')]
    chapter_files = sorted(files_list)  # Sorts numerically if prefixed
    print("Detected chapters:", chapter_files)  # E.g., ['00-Prologue.txt', '01-Awakening.txt', ...]
else:
    print(f"API Miss: {resp.status_code}—check repo path/public status")
    chapter_files = ["00-Prologue.txt"]  # Fallback

Detected chapters: ['00-Prologue.txt', '01-Chapter One.txt', '02-Opening Vibes or a Prelude to a Party.txt', '03-Chapter Two.txt', '04-Mythmaking Monday.txt', '05-Chapter Three.txt', '06-Warehouse Life Episode IV_ Girls.txt', '07-Intermission I.txt', '08-Chapter Four.txt', '09-The Sky is Taking on Light.txt', '10-Chapter Five.txt', "11-You Jump First, I'll Watch.txt", '12-Chapter Six.txt', '13-Feed Me.txt', '14-Chapter Seven.txt', '15-Warehouse Life Episode V_ Career Paths.txt', '16-Intermission II.txt', '17-Chapter Eight.txt', '18-Been Up So Long Looks Like Down To Me.txt', '19-Chapter Nine.txt', '20-Stars Are Shaking.txt', '21-Chapter Ten.txt', '22-Beyond the Ferris Wheel.txt', '23-Chapter Eleven.txt', '24-There Is a World Elsewhere.txt', '25-Chapter Twelve.txt', '26-Warehouse Life Episode VI_ Contact.txt', '27-Intermission III.txt', '28-Go Pawn It on the Mountain.txt', '29-Epilogue.txt', 'full text.txt']


In [15]:
import requests
import json

# Corrected Repo base
REPO_RAW_BASE = "https://raw.githubusercontent.com/babyblueviper1/Viper-Stack-Omega/main/narratives/Waternova/chapters"

# Use auto-detected list from previous cell (or hardcode if API misses)
# chapter_files = ['00-Prologue.txt', '01-Awakening.txt']  # Uncomment if needed

# Auto-fetch function
def load_chapter(file_name):
    url = f"{REPO_RAW_BASE}/{file_name}"
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.text.strip()
    else:
        print(f"Missed: {file_name} (HTTP {resp.status_code}—check name/path)")
        return ""

# Load all
waternova_chapters = {f: load_chapter(f) for f in chapter_files}
waternova_prologue = waternova_chapters.get("00-Prologue.txt", "Fallback: Paste if fetch fails.")

# Bilingual Setup
!pip install googletrans==4.0.0-rc1
from googletrans import Translator
translator = Translator()

# Fuse + Auto-Translate
if 'transcripts' in globals() and transcripts:
    latest_trans = transcripts[0]['transcript']
    fused_en = f"Waternova Prologue: {waternova_prologue[:500]}...\n\nPodcast Resonance: {latest_trans}\n\nEmergent Fusion: Story-logic uplift (GCI >0.7) – Stone eternities entangle prologue voids; pruned 30% motifs."

    # Auto-Spanish
    fused_es = translator.translate(fused_en, dest='es').text

    fusion_dict = {
        'english': fused_en,
        'spanish': fused_es,
        'coherence_proxy': 0.85
    }

    fusion_output = json.dumps(fusion_dict, indent=2)
    with open('bilingual_fusion.json', 'w') as f:
        f.write(fusion_output)
    from google.colab import files
    files.download('bilingual_fusion.json')

    print("Bilingual Cascade: Auto-translated manifest downloaded.")
    print("English Tease:", fused_en[:200] + "...")
    print("Spanish Tease:", fused_es[:200] + "...")
else:
    print("No transcripts yet—run Cells 1-3 first.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bilingual Cascade: Auto-translated manifest downloaded.
English Tease: Waternova Prologue: ﻿#Prologue


Inhale. Exhale. Inhale. Exhale. Inhale. Exhale. Inhale. Exhale . . . 
When I’m like totally done with my very, very relaxing breathing exercises, a pillow of neon-...
Spanish Tease: Prólogo de Waternova: ﻿#Prólogo


Inhala.Exhalar.Inhalar.Exhalar.Inhalar.Exhalar.Inhalar.Exhala...
Cuando termino por completo con mis muy, muy relajantes ejercicios de respiración, una almohada d...
